In [1]:
import os
import time
import numpy as np
import matplotlib.pyplot as plt

In [2]:
### READING DATASET ###

# Paths
dataset_path = 'Dataset'
games_dir = next(os.walk(dataset_path))[1]

tag_to_skip = ['highlights', 'highlights2', '.DS_Store']

# Iterating through games
games_data = []
games_path = [os.path.join(dataset_path, g) for g in games_dir]
for path in games_path:
    game_name = os.path.basename(path)
    if game_name != 'ned_svk':
        game_info = os.listdir(path)
        game_data = dict()
        for info in game_info:
            tag_name = os.path.splitext(info)[0]
            if tag_name not in tag_to_skip:
                csv_file = os.path.join(path, info)
                data_value = np.genfromtxt(csv_file, delimiter=',', dtype=None, filling_values=None)
                game_data[tag_name] = data_value
        game_data['name'] = game_name
        games_data.append(game_data)

games_backup = games_data[:]

In [3]:
games_data = games_backup[:]

In [4]:
### CRIANDO VETOR DE GT ###

normal_cases = ['Normal', 'Inicio', 'Fim']

for game in games_data:
    h = game['highlights_boundaries']
    end = h[-1][0]
    gt = np.zeros(end)
    for line in h[:-1]:
        tag = line[2].decode('UTF-8').split(' ')
        name = [t for t in tag if t != ''][0]
        if name not in normal_cases:
            gt[line[0]-1:line[1]] = 1
    game['gt'] = gt
    del game['highlights_boundaries']

In [5]:
### REMOVENDO VALORES CORRELACIONADOS ###

to_remove = ['em_mcs_energy', 'em_mcs_energy_diff', 'em_mcs_energy_diff_ascending']
for game in games_data:
    for tag in to_remove:
        del game[tag]

In [6]:
### CORTANDO VALORES EXCEDENTES PARA QUE TODOS OS DADOS TENHAM O MESMO TAMANHO ###

for game in games_data:
    values = [len(v) for k, v in game.items() if k not in ['name']]
    min_l = min(values)
    for k, v in game.items():
        if len(v) > min_l:
            game[k] = v[:min_l]
            

### ZERO PADDING ###
max_length = 0
for game in games_data:
    values = [len(v) for k, v in game.items() if k not in ['name']]
    max_length = max(values) if max(values)>max_length else max_length

for game in games_data:
    for k, v in game.items():
        if k not in ['name']:
            zero_v = np.zeros(max_length)
            if len(v) < max_length:
                zero_v[:len(v)] = v
                game[k] = zero_v

In [7]:
### DIVIDING SPLITS FROM TRAIN, EVAL, TEST ###

from sklearn.model_selection import train_test_split

games_name = [g['name'] for g in games_data]
games_train1, games_test = train_test_split(games_name, test_size=0.15)
games_train2, games_val = train_test_split(games_train1, test_size=0.05)

print('Total train: {}'.format(len(games_train2)))
print('Total test: {}'.format(len(games_test)))
print('Total val: {}'.format(len(games_val)))

Total train: 22
Total test: 5
Total val: 2


In [8]:
X_train_dict = {k:[] for k in games_data[0].keys() if k != 'name'}
y_train = []

for game in games_data:
    if game['name'] in games_train1:
        for tag, values in game.items():
            if tag not in ['name']:
                if tag == 'gt':
                    y_train += list(values)
                else:
                    X_train_dict[tag] += list(values)

X_train = X_train_dict.values()


# X_test_dict = {k:[] for k in games_data[0].keys()}
# y_test = []
# for game in games_data:
#     if game['name'] in games_test:
#         for tag, values in game.items():
#             if tag not in ['highlights_boundaries', 'name']:
#                 if tag == 'gt':
#                     y_test += list(values)
#                 else:
#                     X_test_dict[tag] += list(values)
# X_test = []
# for k, v in X_test_dict.items():
#     X_test.append(v)
# del X_test_dict

In [ ]:
X_train

In [66]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification

clf = RandomForestClassifier()
clf.fit(X_train, y_train)

(16, 219949)
(219949,)


ValueError: Number of labels=219949 does not match number of samples=16

In [4]:
games_data[0].keys()

dict_keys(['pc_var_delta', 'em_cs_energy_diff', 'highlights_boundaries', 'dc_percent', 'em_cs_energy_diff_ascending', 'em_st_energy_diff', 'pc_var_theta', 'em_st_energy_diff_ascending', 'em_cs_energy', 'dc_hue_mean', 'gt', 'pm_pitch', 'pm_pitch_diff_ascending', 'pc_rho', 'em_st_energy', 'pc_theta', 'pm_pitch_diff', 'pc_delta', 'name'])